In [29]:
import signals.futures_signals as fs
import signals.utils as su
signals_output = fs.get_futures_butterfly_signals(ticker_list=['LNJ2016','LNN2016','LNV2016'],date_to=20160114)

price_ratio = signals_output['price_ratio']
price_ratio

array([ 0.86200964,  0.85750766,  0.8602261 ,  0.85745394,  0.85608856,
        0.85531629,  0.85361345,  0.85719178,  0.85223426,  0.85784636,
        0.85960265,  0.86857905,  0.86705491,  0.86693548,  0.86248753,
        0.86350099,  0.86252479,  0.86559843,  0.86420157,  0.8575172 ,
        0.85272965,  0.83825481,  0.83829365,  0.83783333,  0.84105298,
        0.83927404,  0.84165593,  0.84314039,  0.84382022,  0.84532605,
        0.84489403,  0.84512705,  0.84705317,  0.84937238,  0.85191808,
        0.84986988,  0.8516537 ,  0.86154835,  0.86219081,  0.86453674])

In [1]:
import opportunity_constructs.futures_butterfly as fb
fb.generate_futures_butterfly_sheet_4date(date_to=20160114)



{'butterflies':      ticker1  ticker2  ticker3 tickerHead tickerClass  trDte1  trDte2  trDte3  \
 0     BK2016   BM2016   BN2016          B      Energy      54      75      96   
 1     BM2016   BN2016   BQ2016          B      Energy      75      96     118   
 2     BN2016   BQ2016   BU2016          B      Energy      96     118     138   
 3     BQ2016   BU2016   BV2016          B      Energy     118     138     160   
 4     BU2016   BV2016   BX2016          B      Energy     138     160     182   
 5     BV2016   BX2016   BZ2016          B      Energy     160     182     203   
 6     BX2016   BZ2016   BF2017          B      Energy     182     203     224   
 7     BZ2016   BF2017   BG2017          B      Energy     203     224     244   
 8     BF2017   BG2017   BH2017          B      Energy     224     244     266   
 9     BG2017   BH2017   BJ2017          B      Energy     244     266     286   
 10    BH2017   BJ2017   BK2017          B      Energy     266     286     309   
 

In [31]:
import signals.utils as su
import shared.statistics as stats
starting_quantile=26
favorable_quantile_move_list=[5,10,15,20,25]
stats.get_number_from_quantile(y=price_ratio,quantile_list=[89,10])



array([ 0.86443954,  0.84087509])

In [22]:
import opportunity_constructs.spread_carry as sc

#spread_carry_output = sc.generate_spread_carry_sheet_4date(report_date=20160128)
#spread_report = spread_carry_output['spread_report']
#ticker_head_list = list(spread_report['tickerHead'].unique())
# Compare BO and S sizing

spread_report[spread_report['tickerHead']==ticker_head_list[19]]
#spread_report[spread_report['tickerHead']=='NG']

,ticker1,ticker2,tickerHead,front_tr_dte,carry,q_carry,reward_risk,price,q,upside,downside,change5,change10,change20
0,CCH2016,CCK2016,CC,32,NaN,NaN,NaN,-6,64,310.0,-10680.6,NaN,NaN,NaN
1,CCK2016,CCN2016,CC,74,-10,0,-0.01,-5,64,172.6,-175.0,NaN,NaN,NaN
2,CCN2016,CCU2016,CC,116,-60,-6,-0.04,1,70,198.8,-193.8,NaN,NaN,NaN
3,CCU2016,CCZ2016,CC,160,-150,-9,-0.12,16,79,144.2,-144.2,NaN,NaN,NaN
4,CCZ2016,CCH2017,CC,223,70,3,0.06,9,76,109.2,-95.0,NaN,NaN,NaN


In [1]:
import opportunity_constructs.curve_pca as cpa
import shared.statistics as stats
import pandas as pd
output = cpa.get_curve_pca_report(ticker_head='ED',date_to = 20160128)

output['pca_results']

,ticker1,ticker2,monthSpread,tr_dte_front,residuals,price,yield,z,factor_load1,factor_load2,change5
0,EDH2016,EDM2016,3,32,-0.027,0.070,0.07,-0.64,-0.272,0.767,NaN
1,EDM2016,EDU2016,3,95,0.010,0.075,0.08,0.30,-0.346,0.325,NaN
2,EDU2016,EDZ2016,3,163,0.026,0.090,0.09,0.74,-0.394,0.129,NaN
3,EDZ2016,EDH2017,3,227,0.015,0.090,0.09,0.40,-0.395,-0.054,NaN
4,EDH2017,EDM2017,3,285,0.007,0.100,0.10,0.18,-0.364,-0.110,NaN
5,EDM2017,EDU2017,3,353,0.005,0.100,0.10,0.21,-0.326,-0.213,NaN
6,EDU2017,EDZ2017,3,416,0.006,0.105,0.11,0.32,-0.289,-0.236,NaN
7,EDZ2017,EDH2018,3,480,-0.014,0.090,0.09,-0.46,-0.257,-0.261,NaN
8,EDH2018,EDM2018,3,543,-0.021,0.090,0.09,-0.55,-0.224,-0.207,NaN
9,EDM2018,EDU2018,3,606,-0.030,0.080,0.08,-0.63,-0.184,-0.197,NaN


In [1]:
import formats.futures_strategy_formats as fsf
fsf.generate_curve_pca_formatted_output()

In [21]:

import matplotlib.pyplot as plt
plt.plot(residuals[:,-2])
plt.grid()         
         
plt.show()





In [27]:
ticker_month_data[-70:,-2]




array([ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12], dtype=int64)

In [26]:
residuals[-70:,-2]

array([ 0.07588417,  0.07882017,  0.08702654,  0.153407  ,  0.14280179,
        0.16753458,  0.17408325,  0.16949982,  0.1137152 ,  0.0924022 ,
        0.11643608, -0.0551888 , -0.06463283,  0.01745819,  0.17526896,
        0.18101742,  0.18272266,  0.17611632,  0.21671111,  0.22102516,
        0.23923634,  0.1492678 ,  0.21741688,  0.17730583,  0.13431045,
        0.14046421,  0.14563187,  0.12043677,  0.13059271,  0.10849144,
        0.15306845,  0.16646527,  0.22326555,  0.22107415,  0.22285868,
        0.3252356 ,  0.30619447,  0.15461736,  0.13789079,  0.15895897,
        0.16309223,  0.11980228,  0.13796355,  0.19093336,  0.24724463,
        0.25552973,  0.19717725,  0.17067874,  0.15090802,  0.08152648,
        0.04576504,  0.05177396, -0.09404087, -0.09990458, -0.1282589 ,
       -0.17673251, -0.19699695, -0.26020227, -0.23956184, -0.37516197,
       -0.46936472, -0.39235337, -0.36365796, -0.44115016, -0.57693024,
       -0.51087076, -0.57080615, -0.63182099, -0.6318899 , -0.55

In [8]:


import matplotlib.pyplot as plt
plt.plot(pca_out['loadings'][1])
plt.grid()         
         
plt.show()



In [2]:
import contract_utilities.expiration as exp
import formats.risk_pnl_formats as rpf
import ta.prepare_daily as prep
report_date = exp.doubledate_shift_bus_days()

rpf.generate_portfolio_pnl_report(as_of_date=report_date)
prep.move_from_dated_folder_2daily_folder(ext='ta', file_name='pnl', folder_date=report_date)


In [3]:
import formats.strategy_followup_formats as sff
import my_sql_routines.my_sql_utilities as msu
import contract_utilities.expiration as exp
report_date = exp.doubledate_shift_bus_days()
con = msu.get_my_sql_connection()
wuhu = sff.generate_futures_butterfly_followup_report(as_of_date=report_date,con=con)
wuhu




,alias,holding_tr_dte,short_tr_dte,z1_initial,z1,QF_initial,QF,total_pnl,downside
14,FCK2016Q2016J2016K2016,1,65,0.86,0.93,97,99,10,-629
6,NGX2016F2017U2016X2016,7,150,2.04,2.23,64,93,-128,-631
2,LCV2016G2017M2016V2016,11,109,1.43,1.45,77,90,-106,-2262
1,RBX2016Z2016V2016X2016,12,173,2.68,3.48,92,80,-1885,-685
8,BG2017H2017F2017G2017,4,216,1.64,1.59,92,79,-242,-2002
7,SN2016Q2016K2016N2016,7,76,1.13,1.04,72,75,3,-986
3,LNZ2016G2017V2016Z2016,9,183,1.86,1.77,90,69,42,-1788
0,CK2016N2016H2016K2016,12,33,1.28,0.91,95,49,115,0
9,SX2016F2017F2017H2017,4,204,-0.71,-0.23,8,43,378,-1204
5,NGM2016N2016N2016Q2016,7,85,-2.48,-1.87,1,36,384,-550


In [5]:
import formats.futures_strategy_formats as fsf
import formats.strategy_followup_formats as sff
import formats.risk_pnl_formats as rpf
import my_sql_routines.futures_price_loader as fpl
import my_sql_routines.my_sql_utilities as msu
import get_price.presave_price as pp
import opportunity_constructs.futures_butterfly as fb
import contract_utilities.expiration as exp
import ta.prepare_daily as prep

sff.generate_futures_butterfly_followup_report(as_of_date=report_date, con=con)
prep.move_from_dated_folder_2daily_folder(ext='ta', file_name='followup', folder_date=report_date)



In [10]:
import risk.historical_risk as hr
import ta.strategy as ts
#aligned_data = hr.get_historical_risk_4strategy(alias='spreadCarryJan16')
n#et_position = ts.get_net_position_4strategy_alias(alias='spreadCarryJan16')
net_position['qty'][2]

NameError: name 'n' is not defined

In [12]:
net_position

,ticker,qty
0,BOF2017,-2
1,BOZ2016,2
4,FCK2016,1
5,FCQ2016,-1
6,KCK2016,-4
7,KCN2016,4
8,KWH2017,2
9,KWZ2016,-2
10,LNG2017,-1
11,LNZ2016,1


In [11]:
[x for x in net_position['ticker']]

['BOF2017',
 'BOZ2016',
 'FCK2016',
 'FCQ2016',
 'KCK2016',
 'KCN2016',
 'KWH2017',
 'KWZ2016',
 'LNG2017',
 'LNZ2016',
 'SK2016',
 'SMV2016',
 'SMZ2016',
 'SN2016']

In [1]:
import my_sql_routines.futures_price_loader as fpl
fpl.update_futures_price_database()







No : 0, RBF2017 loaded
No : 1, RBF2018 loaded
No : 2, RBG2016 loaded
No : 3, RBG2017 loaded
No : 4, RBH2016 loaded
No : 5, RBH2017 loaded
No : 6, RBJ2016 loaded
No : 7, RBJ2017 loaded
No : 8, RBK2016 loaded
No : 9, RBK2017 loaded
No : 10, RBM2016 loaded
No : 11, RBM2017 loaded
No : 12, RBN2016 loaded
No : 13, RBN2017 loaded
No : 14, RBQ2016 loaded
No : 15, RBQ2017 loaded
No : 16, RBU2016 loaded
No : 17, RBU2017 loaded
No : 18, RBV2016 loaded
No : 19, RBV2017 loaded
No : 20, RBX2016 loaded
No : 21, RBX2017 loaded
No : 22, RBZ2016 loaded
No : 23, RBZ2017 loaded
No : 24, SF2016 loaded
No : 25, SF2017 loaded
No : 26, SF2018 loaded
No : 27, SH2016 loaded
No : 28, SH2017 loaded
No : 29, SK2016 loaded
No : 30, SK2017 loaded
No : 31, SN2016 loaded
No : 32, SN2017 loaded
No : 33, SQ2016 loaded
No : 34, SQ2017 loaded
No : 35, SU2016 loaded
No : 36, SU2017 loaded
No : 37, SX2016 loaded
No : 38, SX2017 loaded
No : 39, FCF2016 loaded
Error Loading CME/FCF2017: <class 'Quandl.Quandl.DatasetNotFound'

In [5]:
import get_price.presave_price as pp
pp.generate_and_update_futures_data_file_4tickerhead(ticker_head='CL')








In [5]:
import get_price.curve_data as cd
wuhu = cd.get_rolling_curve_data(ticker_head='CL',num_contracts=12,front_tr_dte_limit=12,date_from=20160125,date_to=20160126)

In [10]:
wuhu

[             cal_dte  change1  change10  change2  change20  change5  change_1  \
 settle_date                                                                     
 2016-01-25        28     0.85       NaN      NaN       NaN      NaN     -1.85   
 2016-01-26        27      NaN       NaN      NaN       NaN      NaN      1.11   
 
              change_5  close_price  cont_indx   ticker ticker_head  \
 settle_date                                                          
 2016-01-25      -0.05        30.34     201603  CLH2016          CL   
 2016-01-26       1.88        31.45     201603  CLH2016          CL   
 
              ticker_month  ticker_year  tr_dte  volume  
 settle_date                                             
 2016-01-25              3         2016      19  598054  
 2016-01-26              3         2016      18  637657  ,
              cal_dte  change1  change10  change2  change20  change5  change_1  \
 settle_date                                                         

In [1]:
import contract_utilities.contract_lists as cl
import my_sql_routines.my_sql_utilities as msu
con = msu.get_my_sql_connection()

cl.get_db_contract_list_filtered(expiration_date_from=20160125,
                                                            expiration_date_to=20200101,
                                                            ticker_head='CL', con=con,ticker_month=6)

[(11657, 'CLM2016', datetime.date(2016, 5, 20)),
 (11658, 'CLM2017', datetime.date(2017, 5, 22)),
 (11659, 'CLM2018', datetime.date(2018, 5, 22)),
 (11660, 'CLM2019', datetime.date(2019, 5, 21))]

In [3]:
import contract_utilities.contract_meta_info as cmi
cmi.futures_contract_months.keys()



dict_keys(['CL', 'SB', 'FC', 'OJ', 'LC', 'KC', 'B', 'RB', 'W', 'LN', 'S', 'C', 'ED', 'KW', 'HO', 'BO', 'CT', 'CC', 'NG', 'SM'])

In [ ]:
np.mean(pnl_path)

In [ ]:
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10
ax.plot(class1_sample[0,:], class1_sample[1,:], class1_sample[2,:],
        'o', markersize=8, color='blue', alpha=0.5, label='class1')
ax.plot(class2_sample[0,:], class2_sample[1,:], class2_sample[2,:],
        '^', markersize=8, alpha=0.5, color='red', label='class2')

plt.title('Samples for class 1 and class 2')
ax.legend(loc='upper right')

plt.show()

In [ ]:
all_samples = np.concatenate((class1_sample, class2_sample), axis=1)
assert all_samples.shape ==(3,40), "The matrix has not the dimensions 3x40"

In [ ]:
mean_x = np.mean(all_samples[0,:])
mean_y = np.mean(all_samples[1,:])
mean_z = np.mean(all_samples[2,:])

mean_vector = np.array([[mean_x],[mean_y],[mean_z]])
print('Mean Vector:\n', mean_vector)






In [ ]:
scatter_matrix = np.zeros((3,3))

for i in range(all_samples.shape[1]):
    scatter_matrix +=(all_samples[:,i].reshape(3,1)).dot(
    (all_samples[:,i].reshape(3,1)-mean_vector).T)
print('Scatter Matrix\n',scatter_matrix)    

In [ ]:
cov_mat = np.cov([all_samples[0,:], all_samples[1,:],all_samples[2,:]])
print('Covariance Matrix:\n', cov_mat)

In [ ]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(scatter_matrix)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)

for i in range(len(eig_val_sc)):
    eigvec_sc = eig_vec_sc[:,i].reshape(1,3).T
    eigvec_cov = eig_vec_cov[:,i].reshape(1,3).T
    assert eigvec_sc.all() == eigvec_cov.all(), 'Eigenvectors are not identical'

    print('Eigenvector {}: \n{}'.format(i+1, eigvec_sc))
    print('Eigenvalue {} from scatter matrix: {}'.format(i+1, eig_val_sc[i]))
    print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))
    print('Scaling factor: ', eig_val_sc[i]/eig_val_cov[i])
    print(40 * '-')

In [ ]:
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,3).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv),
                                         eig_val_sc[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import FancyArrowPatch


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

ax.plot(all_samples[0,:], all_samples[1,:], all_samples[2,:],
        'o', markersize=8, color='green', alpha=0.2)
ax.plot([mean_x], [mean_y], [mean_z],
        'o', markersize=10, color='red', alpha=0.5)
for v in eig_vec_sc.T:
    a = Arrow3D([mean_x, v[0]+mean_x],
                [mean_y, v[1]+mean_y],
                [mean_z, v[2]+mean_z],
                mutation_scale=20, lw=3, arrowstyle="-|>", color="r")

    ax.add_artist(a)
ax.set_xlabel('x_values')
ax.set_ylabel('y_values')
ax.set_zlabel('z_values')

plt.title('Eigenvectors')

plt.show()
multi_columns = pd.MultiIndex.from_product([[ticker1, ticker2, ticker3],data_1.columns.values], names=['first', 'second'])


In [ ]:
transformed = matrix_w.T.dot(all_samples)
assert transformed.shape == (2,40), "The matrix is not 2x40 dimensional."
transformed
matrix_w

In [ ]:
import matplotlib.pyplot as plt
plt.plot(merged_dataframe.index,(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-
         merged_dataframe['BOK2016']['close_price'])
         
         
plt.show()

In [ ]:
(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-merged_dataframe['BOK2016']['close_price']

In [ ]:


import matplotlib.pyplot as plt
plt.plot(range(len(aligned_dataframe.index)),aligned_dataframe['c1']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c2']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c3']['tr_dte'])
plt.grid()         
         
plt.show()

    

    

In [ ]:
merged_dataframe['c1']['tr_dte'].isnull().sum()
merged_dataframe['c3']['tr_dte'].isnull().sum()

In [ ]:
futures_dataframe

In [ ]:
print(trDte1)
print(trDte2)
print(trDte3)

In [ ]:
bf_price = (aligned_dataframe['c1']['close_price']+aligned_dataframe['c3']['close_price'])/2-aligned_dataframe['c2']['close_price']

In [ ]:
plt.plot(range(len(aligned_dataframe.index)),bf_price)
plt.grid()         
         
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(price_ratio)
plt.grid()
plt.show()

In [ ]:
import numpy as np
nan_matrix = np.empty((2,2))
nan_matrix[:] = np.NAN
nan_matrix